## 1. Importing the Data


In [39]:
import pandas as pd
import numpy as np
import datetime as dt

# Load airbnb_price.csv, prices
prices =pd.read_csv("/Users/candicewu/Desktop/airbnb_price.csv")
#review the first few rows
prices.head()


,listing_id,price,nbhood_full
0,2595,225 dollars,"Manhattan, Midtown"
1,3831,89 dollars,"Brooklyn, Clinton Hill"
2,5099,200 dollars,"Manhattan, Murray Hill"
3,5178,79 dollars,"Manhattan, Hell's Kitchen"
4,5238,150 dollars,"Manhattan, Chinatown"


In [40]:
# Load airbnb_room_type.xlsx, xls
xls = pd.ExcelFile("/Users/candicewu/Desktop/airbnb_room_type.xlsx")

# Parse the first sheet from xls, room_types
room_types=xls.parse(0)

#review the first few rows
print(room_types.head())


   listing_id                                description        room_type
0        2595                      Skylit Midtown Castle  Entire home/apt
1        3831            Cozy Entire Floor of Brownstone  Entire home/apt
2        5099  Large Cozy 1 BR Apartment In Midtown East  Entire home/apt
3        5178            Large Furnished Room Near B'way     private room
4        5238         Cute & Cozy Lower East Side 1 bdrm  Entire home/apt


In [41]:
# Load airbnb_last_review.tsv, reviews
reviews =pd.read_csv("/Users/candicewu/Desktop//airbnb_last_review.tsv", sep='\t')# sep='\t' specifies that the separator used in the TSV file is a tab,
print(reviews.head())

   listing_id    host_name   last_review
0        2595     Jennifer   May 21 2019
1        3831  LisaRoxanne  July 05 2019
2        5099        Chris  June 22 2019
3        5178     Shunichi  June 24 2019
4        5238          Ben  June 09 2019


## 2. Cleaning the price column

In [42]:
# Remove whitespace and string characters from prices column
prices["price"] = prices["price"].str.replace(" dollars", "")
prices.head()

,listing_id,price,nbhood_full
0,2595,225,"Manhattan, Midtown"
1,3831,89,"Brooklyn, Clinton Hill"
2,5099,200,"Manhattan, Murray Hill"
3,5178,79,"Manhattan, Hell's Kitchen"
4,5238,150,"Manhattan, Chinatown"


In [43]:
# Convert prices column to numeric datatype
prices["price"] = pd.to_numeric(prices["price"])
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   price        25209 non-null  int64 
 2   nbhood_full  25209 non-null  object
dtypes: int64(2), object(1)
memory usage: 591.0+ KB


In [44]:
# Print descriptive statistics for the price column
print(prices["price"].describe())

count    25209.000000
mean       141.777936
std        147.349137
min          0.000000
25%         69.000000
50%        105.000000
75%        175.000000
max       7500.000000
Name: price, dtype: float64


## 3. Calculating average price

In [45]:
# Subset prices for listings costing $0, free_listings
free_listings = prices[prices["price"]==0]
free_listings.head()


,listing_id,price,nbhood_full
10861,20333471,0,"Bronx, East Morrisania"
10958,20523843,0,"Brooklyn, Bushwick"
11023,20639628,0,"Brooklyn, Bedford-Stuyvesant"
11024,20639792,0,"Brooklyn, Bedford-Stuyvesant"
11025,20639914,0,"Brooklyn, Bedford-Stuyvesant"


In [46]:
# Update prices by removing all free listings and aamaximum price of a night from prices
prices= prices[(prices["price"]!=0)]
print(prices["price"].describe())


count    25202.000000
mean       141.817316
std        147.350650
min         10.000000
25%         69.000000
50%        105.000000
75%        175.000000
max       7500.000000
Name: price, dtype: float64


In [47]:
# Calculate the average price, avg_price
avg_price = prices["price"].mean().round(2)
# Print the average price
print("The average price per night for an Airbnb listing in NYC is ${}.".format(avg_price))


The average price per night for an Airbnb listing in NYC is $141.82.


## 4. Comparing costs to the private rental market

In [48]:
# Add a new column to the prices DataFrame, price_per_month
prices["price_per_month"] = prices["price"] * 365 / 12
prices.head()

,listing_id,price,nbhood_full,price_per_month
0,2595,225,"Manhattan, Midtown",6843.750000
1,3831,89,"Brooklyn, Clinton Hill",2707.083333
2,5099,200,"Manhattan, Murray Hill",6083.333333
3,5178,79,"Manhattan, Hell's Kitchen",2402.916667
4,5238,150,"Manhattan, Chinatown",4562.500000


In [49]:
# Calculate average_price_per_month
average_price_per_month = round(prices["price_per_month"].mean(),2)

# Compare Airbnb and rental market
print("airbnb monthly costs are ${}, while in the private market you would pay {}.".format(average_price_per_month, "$3,100.00"))

airbnb monthly costs are $4313.61, while in the private market you would pay $3,100.00.


## 5. Cleaning the room type column

In [50]:
# Convert the room_type column to lowercase
room_types["room_type"] = room_types["room_type"].str.lower()
room_types.head()

,listing_id,description,room_type
0,2595,Skylit Midtown Castle,entire home/apt
1,3831,Cozy Entire Floor of Brownstone,entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,entire home/apt
3,5178,Large Furnished Room Near B'way,private room
4,5238,Cute & Cozy Lower East Side 1 bdrm,entire home/apt


In [51]:
# Update the room_type column to category data type
room_types["room_type"] = room_types["room_type"].astype("category")
room_types.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   listing_id   25209 non-null  int64   
 1   description  25199 non-null  object  
 2   room_type    25209 non-null  category
dtypes: category(1), int64(1), object(1)
memory usage: 418.8+ KB


In [52]:
# Create the variable room_frequencies
room_frequencies = room_types["room_type"].value_counts()

# Print room_frequencies
print(room_frequencies)

entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64


## 6. What timeframe are we working with?

In [53]:
# Change the data type of the last_review column to datetime
reviews["last_review"] = pd.to_datetime(reviews["last_review"])
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   listing_id   25209 non-null  int64         
 1   host_name    25201 non-null  object        
 2   last_review  25209 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 591.0+ KB


In [54]:
# Create first_reviewed, the earliest review date
first_reviewed = reviews["last_review"].dt.date.min()
# Create last_reviewed, the most recent review date
last_reviewed = reviews["last_review"].dt.date.max()

# Print the oldest and newest reviews from the DataFrame
print("The earliest Airbnb review is {}, the latest review is {}".format(first_reviewed, last_reviewed))

The earliest Airbnb review is 2019-01-01, the latest review is 2019-07-09


## 7. Joining the DataFrames.

In [55]:
# Merge prices and room_types to create rooms_and_prices
rooms_and_prices = pd.merge(prices, room_types, how='outer', on="listing_id")
rooms_and_prices.head()


,listing_id,price,nbhood_full,price_per_month,description,room_type
0,2595,225.0,"Manhattan, Midtown",6843.750000,Skylit Midtown Castle,entire home/apt
1,3831,89.0,"Brooklyn, Clinton Hill",2707.083333,Cozy Entire Floor of Brownstone,entire home/apt
2,5099,200.0,"Manhattan, Murray Hill",6083.333333,Large Cozy 1 BR Apartment In Midtown East,entire home/apt
3,5178,79.0,"Manhattan, Hell's Kitchen",2402.916667,Large Furnished Room Near B'way,private room
4,5238,150.0,"Manhattan, Chinatown",4562.500000,Cute & Cozy Lower East Side 1 bdrm,entire home/apt


In [56]:
# Merge rooms_and_prices with the reviews DataFrame to create airbnb_merged
airbnb_merged = rooms_and_prices.merge(reviews, how="outer", on="listing_id")
airbnb_merged.head()


,listing_id,price,nbhood_full,price_per_month,description,room_type,host_name,last_review
0,2595,225.0,"Manhattan, Midtown",6843.750000,Skylit Midtown Castle,entire home/apt,Jennifer,2019-05-21
1,3831,89.0,"Brooklyn, Clinton Hill",2707.083333,Cozy Entire Floor of Brownstone,entire home/apt,LisaRoxanne,2019-07-05
2,5099,200.0,"Manhattan, Murray Hill",6083.333333,Large Cozy 1 BR Apartment In Midtown East,entire home/apt,Chris,2019-06-22
3,5178,79.0,"Manhattan, Hell's Kitchen",2402.916667,Large Furnished Room Near B'way,private room,Shunichi,2019-06-24
4,5238,150.0,"Manhattan, Chinatown",4562.500000,Cute & Cozy Lower East Side 1 bdrm,entire home/apt,Ben,2019-06-09


In [57]:
# check the number of missing values
airbnb_merged.isnull().sum()

listing_id          0
price               7
nbhood_full         7
price_per_month     7
description        10
room_type           0
host_name           8
last_review         0
dtype: int64

In [58]:
# Drop missing values from airbnb_merged
airbnb_merged.dropna(inplace=True)
airbnb_merged.isnull().sum()

listing_id         0
price              0
nbhood_full        0
price_per_month    0
description        0
room_type          0
host_name          0
last_review        0
dtype: int64

In [59]:
# Check if there are any duplicate values
print("There are {} duplicates in the DataFrame.".format(airbnb_merged.duplicated().sum()))

There are 0 duplicates in the DataFrame.


## 8. Analyzing listing prices by NYC borough

In [60]:
# Extract information from the nbhood_full column and store as a new column, borough
airbnb_merged["borough"] = airbnb_merged["nbhood_full"].str.partition(',')[0]
airbnb_merged.head()

,listing_id,price,nbhood_full,price_per_month,description,room_type,host_name,last_review,borough
0,2595,225.0,"Manhattan, Midtown",6843.750000,Skylit Midtown Castle,entire home/apt,Jennifer,2019-05-21,Manhattan
1,3831,89.0,"Brooklyn, Clinton Hill",2707.083333,Cozy Entire Floor of Brownstone,entire home/apt,LisaRoxanne,2019-07-05,Brooklyn
2,5099,200.0,"Manhattan, Murray Hill",6083.333333,Large Cozy 1 BR Apartment In Midtown East,entire home/apt,Chris,2019-06-22,Manhattan
3,5178,79.0,"Manhattan, Hell's Kitchen",2402.916667,Large Furnished Room Near B'way,private room,Shunichi,2019-06-24,Manhattan
4,5238,150.0,"Manhattan, Chinatown",4562.500000,Cute & Cozy Lower East Side 1 bdrm,entire home/apt,Ben,2019-06-09,Manhattan


In [61]:
# Group by borough and calculate summary statistics
boroughs = airbnb_merged.groupby("borough")["price"].agg(["sum", "mean", "median", "count"])
boroughs.head()

,sum,mean,median,count
borough,,,,
Bronx,55156.0,79.247126,65.0,696
Brooklyn,1275250.0,122.021816,95.0,10451
Manhattan,1898417.0,184.044304,149.0,10315
Queens,320715.0,92.826339,70.0,3455
Staten Island,22974.0,86.044944,71.0,267


In [62]:
# Round boroughs to 2 decimal places, and sort by mean in descending order
boroughs = boroughs.round(2).sort_values(by="mean", ascending=False)

# Print boroughs
print(boroughs)

                     sum    mean  median  count
borough                                        
Manhattan      1898417.0  184.04   149.0  10315
Brooklyn       1275250.0  122.02    95.0  10451
Queens          320715.0   92.83    70.0   3455
Staten Island    22974.0   86.04    71.0    267
Bronx            55156.0   79.25    65.0    696


## 9. Price range by borough

In [63]:
# Create labels for the price range, label_names
label_names = ["Budget", "Average", "Expensive", "Extravagant"]
# Create the label ranges, ranges
ranges = [0, 69, 175, 350, np.inf]
# Insert new column, price_range, into DataFrame
airbnb_merged["price_range"] = pd.cut(airbnb_merged['price'], bins=ranges, labels=label_names)

# Calculate borough and price_range frequencies, prices_by_borough
prices_by_borough = airbnb_merged.groupby(['borough', 'price_range']).size()
print(prices_by_borough)

borough        price_range
Bronx          Budget          381
               Average         285
               Expensive        25
               Extravagant       5
Brooklyn       Budget         3194
               Average        5532
               Expensive      1466
               Extravagant     259
Manhattan      Budget         1148
               Average        5285
               Expensive      3072
               Extravagant     810
Queens         Budget         1631
               Average        1505
               Expensive       291
               Extravagant      28
Staten Island  Budget          124
               Average         123
               Expensive        20
               Extravagant       0
dtype: int64
